In [1]:
# import numpy pandas etc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [5]:
# load twitter data
data = pd.read_csv("sent.csv", header=None, encoding='utf-8')


In [7]:
# data columns:
# 0 - sentiment (0 = negative, 2 = neutral, 4 = positive)
# 1 - id of tweet
# 2 - date of tweet
# 3 - query
# 4 - user
# 5 - text
# change column names
data.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']


In [75]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, SimpleRNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [10]:
# Preprocess the data
processedData = data[['sentiment', 'text']]
processedData['sentiment'] = processedData['sentiment'].replace({4: 1})  # Convert positive sentiment to 1

/tmp/ipykernel_820/155909423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processedData['sentiment'] = processedData['sentiment'].replace({4: 1})  # Convert positive sentiment to 1


In [21]:
# move sentiment to -1,0,1
processedData['sentiment'] = processedData['sentiment'].replace({0: -1})  # Convert negative sentiment to -1
processedData['sentiment'] = processedData['sentiment'].replace({2: 0})  # Convert neutral sentiment to 0

/tmp/ipykernel_820/520913739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processedData['sentiment'] = processedData['sentiment'].replace({0: -1})  # Convert negative sentiment to -1
/tmp/ipykernel_820/520913739.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processedData['sentiment'] = processedData['sentiment'].replace({2: 0})  # Convert neutral sentiment to 0


In [15]:
# Split into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
vocab_size = len(tokenizer.word_index) + 1

# Convert text sequences to numerical sequences
train_sequences = tokenizer.texts_to_sequences(train_data['text'])
test_sequences = tokenizer.texts_to_sequences(test_data['text'])

# Pad the sequences to have the same length
max_sequence_length = 100  # Set the desired sequence length
train_data_padded = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_data_padded = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Convert the target labels to categorical
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data['sentiment'])
test_labels = label_encoder.transform(test_data['sentiment'])

In [22]:
# Create the LSTM model
embedding_dim = 100  # Set the embedding dimension
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='tanh'))

In [23]:
# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_data_padded, train_labels, validation_data=(test_data_padded, test_labels), epochs=50, batch_size=32)


Epoch 1/50
13/13 [==============================] - 4s 139ms/step - loss: 1.7819 - accuracy: 0.3291 - val_loss: -1.0979 - val_accuracy: 0.3400
Epoch 2/50
13/13 [==============================] - 1s 115ms/step - loss: 0.1808 - accuracy: 0.2638 - val_loss: -0.9770 - val_accuracy: 0.3400
Epoch 3/50
13/13 [==============================] - 1s 115ms/step - loss: 0.1150 - accuracy: 0.2638 - val_loss: -0.7888 - val_accuracy: 0.3400
Epoch 4/50
13/13 [==============================] - 2s 121ms/step - loss: -0.0087 - accuracy: 0.2638 - val_loss: -0.6013 - val_accuracy: 0.3400
Epoch 5/50
13/13 [==============================] - 2s 118ms/step - loss: -0.1523 - accuracy: 0.2638 - val_loss: -0.7267 - val_accuracy: 0.3400
Epoch 6/50
13/13 [==============================] - 1s 107ms/step - loss: -0.5991 - accuracy: 0.2739 - val_loss: -1.1272 - val_accuracy: 0.3400
Epoch 7/50
13/13 [==============================] - 1s 106ms/step - loss: -2.0318 - accuracy: 0.3769 - val_loss: -1.8215 - val_accuracy: 0.

In [24]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_data_padded, test_labels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

4/4 [==============================] - 0s 25ms/step - loss: -3.1687 - accuracy: 0.4600
Test Loss: -3.1687488555908203
Test Accuracy: 0.46000000834465027


In [30]:
# Take 5 random tweets and their labels from the processedData
sample = processedData.sample(5)
sample_labels = sample['sentiment'].values.tolist()
sample_sequences = tokenizer.texts_to_sequences(sample['text'])
sample_padded = pad_sequences(sample_sequences, maxlen=max_sequence_length)
toPredict = processedData.sample(1)


In [31]:
# print
print(sample)
print(sample_labels)

     sentiment                                               text
161         -1  Recovering from surgery..wishing @julesrenner ...
444          0  This is cold.. I was looking at google's chart...
165         -1  @kirstiealley my dentist is great but she's ex...
475          0  has a date with bobby flay and gut fieri from ...
412          0  First dentist appointment [in years] on Wednes...
[-1, 0, -1, 0, 0]


0----------------------------

In [48]:
# print all data where user is 'chadfu'
# data['user']
# print unique
# print(data['user'].unique())
# print unique users and how many tweets
print(data['user'].value_counts())
# print(data.loc[data['user'] == 'chadfu'])

vmkobs             3
MamiYessi          2
tradecruz          2
SimpleManJess      2
souleaterjh        2
                  ..
sinostream         1
drlombardo         1
myfoxdc            1
BreakingBizNews    1
captain_pete       1
Name: user, Length: 490, dtype: int64


In [88]:
import random
import numpy as np
import pandas as pd

random.seed(42)

# Function to generate a single instance
def generate_instance():
    length = random.randint(1, 10)  # Random length of the instance
    numbers = random.choices([-1, 0, 1], k=length)  # Random numbers (-1, 0, or 1)
    majority = np.sign(np.sum(numbers))  # Majority label
    return numbers, majority

# Generate instances and collect them in a list
instances = []
for _ in range(100000):
    numbers, majority = generate_instance()
    instances.append((numbers, majority))

# Convert the list to a DataFrame
data = pd.DataFrame(instances, columns=["Numbers", "Majority"])

# Save the dataset to a CSV file
data.to_csv("synthetic_dataset.csv", index=False)


In [92]:
# modify Numbers to be string
data['Numbers'] = data['Numbers'].astype(str)

In [89]:
# transform Majority to dummies
data = pd.get_dummies(data, columns=['Majority'])

In [94]:
data

,Numbers,Majority_-1,Majority_0,Majority_1
0,-1 -1,1,0,0
1,-1 -1 1 0,1,0,0
2,0 -1 -1 0 -1 -1 0 0 -1 0,1,0,0
3,1,0,0,1
4,1 0 -1,0,1,0
...,...,...,...,...
99995,0,0,1,0
99996,-1 0 1 0 1 1 1 0 1,0,0,1
99997,-1 0 -1 0,1,0,0
99998,-1 -1,1,0,0


In [95]:

# Convert the target labels to categorical
train_majority = train_data[['Majority_-1', 'Majority_0', 'Majority_1']].values
test_majority = test_data[['Majority_-1', 'Majority_0', 'Majority_1']].values


In [93]:
# Convert the numbers column to a string
data["Numbers"] = data["Numbers"].apply(eval).apply(lambda x: ' '.join(map(str, x)))

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data["Numbers"])
vocab_size = len(tokenizer.word_index) + 1

# Convert text sequences to numerical sequences
train_sequences = tokenizer.texts_to_sequences(train_data["Numbers"])
test_sequences = tokenizer.texts_to_sequences(test_data["Numbers"])

# Pad the sequences to have the same length
max_sequence_length = max(len(seq) for seq in train_sequences + test_sequences)
train_data_padded = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_data_padded = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Convert the majority column to an array
train_majority = train_data["Majority"].values
test_majority = test_data["Majority"].values



KeyError: 'Majority'

In [71]:
# Create the LSTM model
embedding_dim = 64  # Set the embedding dimension
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="tanh"))

In [96]:
model = Sequential()
model.add(LSTM(128, input_shape=(None, 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

2023-06-03 22:19:20.403616: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-03 22:19:20.404934: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-03 22:19:20.406790: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [97]:
# Compile and train the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_data_padded, train_majority, validation_data=(test_data_padded, test_majority), epochs=50, batch_size=32)

Epoch 1/50


2023-06-03 22:19:27.479926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-03 22:19:27.482950: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-03 22:19:27.486183: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2498/2500 [============================>.] - ETA: 0s - loss: 0.5976 - accuracy: 0.4154

2023-06-03 22:20:10.768880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-03 22:20:10.770633: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-03 22:20:10.771882: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2500/2500 [==============================] - 48s 18ms/step - loss: 0.5976 - accuracy: 0.4154 - val_loss: 0.5525 - val_accuracy: 0.4600
Epoch 2/50
2500/2500 [==============================] - 44s 18ms/step - loss: 0.5188 - accuracy: 0.4793 - val_loss: 0.5469 - val_accuracy: 0.4635
Epoch 3/50
2500/2500 [==============================] - 47s 19ms/step - loss: 0.5133 - accuracy: 0.4781 - val_loss: 0.5111 - val_accuracy: 0.4772
Epoch 4/50
2500/2500 [==============================] - 44s 18ms/step - loss: 0.5130 - accuracy: 0.4778 - val_loss: 0.5107 - val_accuracy: 0.4816
Epoch 5/50
2500/2500 [==============================] - 47s 19ms/step - loss: 0.5123 - accuracy: 0.4806 - val_loss: 0.5103 - val_accuracy: 0.4852
Epoch 6/50
2500/2500 [==============================] - 45s 18ms/step - loss: 0.5119 - accuracy: 0.4814 - val_loss: 0.5103 - val_accuracy: 0.4838
Epoch 7/50
 682/2500 [=======>......................] - ETA: 31s - loss: 0.5134 - accuracy: 0.4779

KeyboardInterrupt: 

In [98]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_data_padded, test_majority)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

625/625 [==============================] - 5s 8ms/step - loss: 0.5097 - accuracy: 0.4868
Test Loss: 0.5096907019615173
Test Accuracy: 0.4867500066757202


In [100]:
# Take 1 random example from the test set, predict it then show real label
sample = test_data.sample(1)
sample_number = sample["Numbers"].values.tolist()
sample_majority = sample[["Majority_-1", "Majority_0", "Majority_1"]].values.tolist()
sample_sequence = tokenizer.texts_to_sequences(sample_number)
sample_padded = pad_sequences(sample_sequence, maxlen=max_sequence_length)
prediction = model.predict(sample_padded)
print("Number:", sample_number)
print("Real Label:", sample_majority)
print("Predicted Label:", prediction)

1/1 [==============================] - 0s 27ms/step
Number: ['1']
Real Label: [[0, 0, 1]]
Predicted Label: [[4.8005041e-01 3.7434061e-06 5.1994586e-01]]


In [102]:
# take the max of the prediction, show it as dummies
print("Predicted Label:", np.argmax(prediction, axis=1))

Predicted Label: [2]
